# Phase 4 — FSM Navigation

Integrate perception, memory, and FSM to produce navigation commands.

## Imports and setup

In [ ]:
from pathlib import Path
import sys

import cv2
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

from src.utils import load_config, ensure_demo_video, open_capture, iter_frames, release_capture
from src.perception_flow import compute_optical_flow, compute_obstacle_risk, compute_openness
from src.memory import NavigationMemory
from src.fsm import NavigationFSM
from src.controller import NavigationController

config = load_config("configs/default.yaml")
video_path = ensure_demo_video(config)
print(f"Video: {video_path}")

## Full navigation loop

In [ ]:
cap = open_capture(config)
max_frames = config["video"].get("max_frames") or 200

# Initialize components
memory = NavigationMemory(config)
fsm = NavigationFSM(config)
controller = NavigationController(config)

# Storage
results = {
    "risk_smooth": [],
    "openness_smooth": [],
    "state": [],
    "command": [],
    "linear_v": [],
    "angular_w": [],
}

prev_gray = None

for idx, frame in tqdm(iter_frames(cap, max_frames=max_frames, convert_rgb=False), total=max_frames):
    curr_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    if prev_gray is not None:
        # Perception
        flow = compute_optical_flow(prev_gray, curr_gray, config)
        risk_raw = compute_obstacle_risk(flow, config)
        openness_raw, direction = compute_openness(flow, config)
        
        # Memory
        signals = memory.update(risk_raw, openness_raw, direction)
        
        # FSM
        state = fsm.update(signals)
        
        # Controller
        recovery_dir = fsm.get_recovery_direction()
        preferred_dir = recovery_dir if recovery_dir else signals["preferred_direction"]
        command = controller.get_command(state, preferred_dir)
        
        # Store
        results["risk_smooth"].append(signals["risk_smooth"])
        results["openness_smooth"].append(signals["openness_smooth"])
        results["state"].append(state)
        results["command"].append(command.command_name)
        results["linear_v"].append(command.linear_velocity)
        results["angular_w"].append(command.angular_velocity)
    
    prev_gray = curr_gray

release_capture(cap)

print(f"Processed {len(results['state'])} frames")

## Analyze state distribution

In [ ]:
from collections import Counter

state_counts = Counter(results["state"])
total_frames = len(results["state"])

print("=" * 50)
print("State Distribution")
print("=" * 50)
for state, count in state_counts.most_common():
    pct = 100.0 * count / total_frames
    print(f"{state:20s}: {count:4d} frames ({pct:5.1f}%)")
print("=" * 50)

# Count state transitions
transitions = 0
for i in range(1, len(results["state"])):
    if results["state"][i] != results["state"][i-1]:
        transitions += 1

print(f"Total state transitions: {transitions}")
print(f"Avg frames per state: {total_frames / len(state_counts):.1f}")

## Visualize FSM behavior

In [ ]:
frames = np.arange(len(results["state"]))

# Map states to numeric values for plotting
state_map = {
    "CRUISE_FORWARD": 0,
    "AVOID_LEFT": 1,
    "AVOID_RIGHT": 2,
    "STOP_TOO_CLOSE": 3,
    "RECOVERY": 4,
}
state_values = [state_map.get(s, -1) for s in results["state"]]

fig, axes = plt.subplots(4, 1, figsize=(14, 12), sharex=True)

# Risk
axes[0].plot(frames, results["risk_smooth"], label="Smoothed Risk", color="red")
axes[0].axhline(config["risk"]["thresholds"]["avoid"], color="orange", linestyle="--", label="Avoid")
axes[0].axhline(config["risk"]["thresholds"]["stop"], color="darkred", linestyle="--", label="Stop")
axes[0].set_ylabel("Risk Score")
axes[0].set_ylim([0, 1])
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Openness
axes[1].plot(frames, results["openness_smooth"], label="Smoothed Openness", color="blue")
axes[1].axhline(0, color="black", linestyle="-", linewidth=0.5)
axes[1].set_ylabel("Openness (L-R)")
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# FSM State
axes[2].plot(frames, state_values, drawstyle="steps-post", color="purple", linewidth=2)
axes[2].set_ylabel("State")
axes[2].set_yticks(list(state_map.values()))
axes[2].set_yticklabels(list(state_map.keys()))
axes[2].grid(True, alpha=0.3)
axes[2].set_title("FSM State Transitions")

# Motion commands
axes[3].plot(frames, results["linear_v"], label="Linear Velocity", color="green")
axes[3].plot(frames, results["angular_w"], label="Angular Velocity", color="orange")
axes[3].axhline(0, color="black", linestyle="-", linewidth=0.5)
axes[3].set_ylabel("Velocity")
axes[3].set_xlabel("Frame")
axes[3].legend()
axes[3].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Check for flip-flop behavior

In [ ]:
# Look for rapid state oscillations
rapid_flips = 0
window = 5

for i in range(window, len(results["state"])):
    recent_states = results["state"][i-window:i]
    # Count unique states in window
    unique_states = len(set(recent_states))
    if unique_states >= 3:
        rapid_flips += 1

print(f"Rapid state changes (>= 3 different states in {window} frames): {rapid_flips}")

if rapid_flips < 10:
    print("✓ FSM is stable - minimal flip-flopping")
else:
    print("⚠ FSM shows some instability - consider tuning thresholds")

# Check recovery activations
recovery_count = sum(1 for s in results["state"] if s == "RECOVERY")
print(f"\nRecovery mode activated: {recovery_count} frames")
if recovery_count > 0:
    print("✓ Anti-oscillation recovery is working")

## Verification

Phase 4 verification:
- FSM produces stable state transitions
- Commands are generated correctly for each state
- Recovery mode activates when oscillation is detected
- Minimal rapid flip-flopping between states